In [1]:
# Import Libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [11]:
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:
train_set = datasets.ImageFolder("C:/Users/EGYPT_LAPTOP/Desktop/chest_xray/train", transform = transformations)
test_set = datasets.ImageFolder("C:/Users/EGYPT_LAPTOP/Desktop/chest_xray/test", transform = transformations)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32,drop_last = True, shuffle=True,num_workers=3)
val_loader  = torch.utils.data.DataLoader(test_set, batch_size =32,drop_last = True, shuffle=True,num_workers=3)

In [5]:
model = models.densenet161(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [6]:

classifier_input = model.classifier.in_features
num_labels = 
classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, num_labels),
                           nn.LogSoftmax(dim=1))
# Replace default classifier with new classifier
model.classifier = classifier

In [12]:
classifier_input

2208

In [7]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device specified above
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [9]:
# Set the error function using torch.nn as nn library
weight = torch.Tensor([0.74,0.24])
criterion = nn.NLLLoss(weight= weight).to(device)
# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.classifier.parameters())

In [10]:
epochs = 20
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    
    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
       
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
       
        output = model.forward(inputs)
       
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
     
        train_loss += loss.item()*inputs.size(0)
        
       
        counter += 1
        print(counter, "/", len(train_loader))
        
    # Evaluating the model
    model.eval()
    counter = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            output = model.forward(inputs)
            valloss = criterion(output, labels)
            val_loss += valloss.item()*inputs.size(0)
           
            output = torch.exp(output)
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            counter += 1
            print(counter, "/", len(val_loader))
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = val_loss/len(val_loader.dataset)
    print('Accuracy: ', accuracy/len(val_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

C:\Users\EGYPT_LAPTOP\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1 / 163
2 / 163
3 / 163
4 / 163
5 / 163
6 / 163
7 / 163
8 / 163
9 / 163
10 / 163
11 / 163
12 / 163
13 / 163
14 / 163
15 / 163
16 / 163
17 / 163
18 / 163
19 / 163
20 / 163
21 / 163
22 / 163
23 / 163
24 / 163
25 / 163
26 / 163
27 / 163
28 / 163
29 / 163
30 / 163
31 / 163
32 / 163
33 / 163
34 / 163
35 / 163
36 / 163
37 / 163
38 / 163
39 / 163
40 / 163
41 / 163
42 / 163
43 / 163
44 / 163
45 / 163
46 / 163
47 / 163
48 / 163
49 / 163
50 / 163
51 / 163
52 / 163
53 / 163
54 / 163
55 / 163
56 / 163
57 / 163
58 / 163
59 / 163
60 / 163
61 / 163
62 / 163
63 / 163
64 / 163
65 / 163
66 / 163
67 / 163
68 / 163
69 / 163
70 / 163
71 / 163
72 / 163
73 / 163
74 / 163
75 / 163
76 / 163
77 / 163
78 / 163
79 / 163
80 / 163
81 / 163
82 / 163
83 / 163
84 / 163
85 / 163
86 / 163
87 / 163
88 / 163
89 / 163
90 / 163
91 / 163
92 / 163
93 / 163
94 / 163
95 / 163
96 / 163
97 / 163
98 / 163
99 / 163
100 / 163
101 / 163
102 / 163
103 / 163
104 / 163
105 / 163
106 / 163
107 / 163
108 / 163
109 / 163
110 / 163
111 / 16